In [10]:
import pandas as pd
import numpy as np
import json
import os
import io
import re
import fitz
import ast
import urllib.request as urllib2

In [11]:
class pdf_reader():
  def __init__(self,**kwargs):
    #super().__init__(**kwargs)
    for key, value in kwargs.items():
      if key=='file':
        self.file_path = value
    self.output_io = io.StringIO()
    self.lst = []

  def text_extract(self):
    f = fitz.open(self.file_path)
    output_io = io.StringIO()
    for pg in f:
      txt = pg.get_text('blocks',0)
      output_io.write(str(txt))
    return output_io

  def to_list(self,txt):
    lst_str = [ m.start() for m in re.finditer('\[\(',txt)]
    lst_end = [ m.start()+2 for m in re.finditer('\)\]',txt)]
    lst = [ast.literal_eval(txt[i[0]:i[1]]) for i in zip(lst_str,lst_end)]
    return lst

  def scrape_file(self):
    self.output_io = self.text_extract()
    lst = self.to_list(self.output_io.getvalue())
    self.lst = lst
    return len(lst)


In [12]:
class fek(pdf_reader):

  def __init__(self,**kwargs):
    super().__init__(**kwargs)
    self.fek = ''
    self.date = ''
    self.fek_type = ''
    self.number = np.nan
    self.title = ''
    self.president_order_references = []
    self.law_references = []
    self.constitution_references = []

  def proceess_fek(self):

    # Get fek type and number. Fill in only if not already filled in
    for page in self.lst:
      for txt_block in page:
        if bool('νομοθετικου περιεχομενου' in txt_block[4].lower()):
          if self.fek_type == '': self.fek_type = 'πραξη νομοθετικου περιεχομενου'
        if bool('προεδρικο διαταγμα' in txt_block[4].lower()):
          if self.fek_type == '': self.fek_type = 'προεδρικο διαταγμα'
          if np.isnan(self.number): self.number = self.get_number(txt_block[4].lower())
        if bool('πραξεις υπουργικου συμβουλιου' in txt_block[4].lower()):
          if self.fek_type == '': self.fek_type = 'πραξεις υπουργικου συμβουλιου'
        if bool('nomoς' in txt_block[4].lower()):
          if self.fek_type == '': self.fek_type = 'νομος'
          if np.isnan(self.number): self.number = self.get_number(txt_block[4].lower())
        if bool('ΕΦΗΜΕΡΙ∆Α TΗΣ ΚΥΒΕΡΝΗΣΕΩΣ' in txt_block[4]):
          if self.fek == '': self.fek_number_n_date(txt_block[4])

        # add references
        presdential, law, constitution = self.find_refrences(txt_block[4])
        if presdential:
          self.president_order_references = list(set(self.president_order_references).union(set(presdential)))
        if law:
          self.law_references = list(set(self.law_references).union(set(law)))
        if constitution:
          self.constitution_references = list(set(self.constitution_references).union(set(constitution)))


    # Title is in first block of first page
    self.title = " ".join(str(self.lst[0][1][4]).replace('\n','').split())



  def get_number(self,txt:str):
    m = re.search('αριθμ.',txt)
    from_n = m.span()[1]
    return int(txt[from_n:])

  def find_refrences(self,txt:str):
    # presidential order
    presidential_order_pattern = "π\.δ\. [0-9]+/[0-9]+"
    presidential_order_matches = re.findall(presidential_order_pattern, txt)
    # law order
    law_pattern = "ν\. [0-9]+/[0-9]+"
    law_matches = re.findall(law_pattern, txt)
    # constitution
    constitution_pattern = "άρθρου [0-9]+ του Συντάγματος" #"((την παρ\.|Την παρ\.|τις παρ\.|Τις παρ\.) ([0-9]+,)* [0-9]+( και [0-9]+)? του άρθρου [0-9]+(, )?)*και (την παρ\.|τις παρ\.) ([0-9]+,)* [0-9]+( και [0-9]+)? του άρθρου [0-9]+ του Συντάγματος"
    constitution_matches = re.findall(constitution_pattern, txt)

    return presidential_order_matches, law_matches, constitution_matches

  def fek_number_n_date(self,txt):
    m = re.search('/',txt)
    from_n = m.span()[1]
    self.date = txt[from_n:].replace('\n','')
    self.fek = ' '.join(txt[:(from_n-1)].split('\n')[::2]).strip() # txt[:(from_n-1)].replace('\n',' ')
    return

  def info(self,prnt=False):
      fek_dict = {
          'file': self.file_path,
          'fek' : self.fek,
          'date': self.date,
          'type': self.fek_type,
          'number': self.number,
          'title': self.title,
          'presidential orders ref': self.president_order_references,
          'law ref': self.law_references,
          'constitution ref': self.constitution_references
          }
      if prnt == True:
        for key, val in fek_dict.items():
          print(key,':',val)
        return
      return fek_dict

  def _toDataFrame(self):
    fek_dict = self.info()
    for key,val in self.info().items():
      if type(val) is list:
        fek_dict[key] = ', '.join(val)
    return pd.DataFrame.from_dict([fek_dict])



In [13]:
files = os.listdir()
files = [ f for f in files if '.pdf' in f]
files = [ f for f in files if 'Tefxos A' in f]

In [14]:
results = pd.DataFrame()

In [15]:
succeded = 0
failed =0
for f in files:
  #print(f'Processing {f}')
  try:  
    my_fek = fek(file=f)
    my_fek.scrape_file()
    my_fek.proceess_fek()
    if len(results) == 0 :
      results = my_fek._toDataFrame()
    else:
      results = pd.concat([results, my_fek._toDataFrame()], ignore_index=True, sort=False)
    succeded = succeded+1
    print(f'Finished {f}')
  except: 
    print(f'Failed {f}')
    failed = failed + 1
print(f'\n  ------------------'*5)
print(f'Finished processing {len(files)} FEKs\nSucceded: {succeded}\nFailed: {failed}')

Finished FEK-2021-Tefxos A-00001-downloaded -21_12_2023.pdf
Finished FEK-2021-Tefxos A-00002-downloaded -21_12_2023.pdf
Finished FEK-2021-Tefxos A-00003-downloaded -21_12_2023.pdf
Failed FEK-2021-Tefxos A-00004-downloaded -21_12_2023.pdf
Failed FEK-2021-Tefxos A-00005-downloaded -21_12_2023.pdf
Finished FEK-2021-Tefxos A-00006-downloaded -21_12_2023.pdf
Finished FEK-2021-Tefxos A-00007-downloaded -21_12_2023.pdf
Finished FEK-2021-Tefxos A-00008-downloaded -21_12_2023.pdf
Finished FEK-2021-Tefxos A-00009-downloaded -21_12_2023.pdf
Finished FEK-2021-Tefxos A-00010-downloaded -21_12_2023.pdf
Failed FEK-2021-Tefxos A-00011-downloaded -21_12_2023.pdf
Finished FEK-2021-Tefxos A-00012-downloaded -21_12_2023.pdf
Finished FEK-2021-Tefxos A-00013-downloaded -21_12_2023.pdf
Finished FEK-2021-Tefxos A-00014-downloaded -21_12_2023.pdf
Finished FEK-2021-Tefxos A-00015-downloaded -21_12_2023.pdf
Failed FEK-2021-Tefxos A-00016-downloaded -21_12_2023.pdf
Finished FEK-2021-Tefxos A-00017-downloaded -21_

In [16]:
results.shape

(585, 9)

In [17]:
results.head(20)

,file,fek,date,type,number,title,presidential orders ref,law ref,constitution ref
0,FEK-2021-Tefxos A-00001-downloaded -21_12_2023...,ΕΦΗΜΕΡΙ∆Α TΗΣ ΚΥΒΕΡΝΗΣΕΩΣ Τεύχος A’ 1,05.01.2021,προεδρικο διαταγμα,1.0,"Αποδοχή παραίτησης Υπουργών, Αναπληρωτή Υπουργ...",π.δ. 29/2018,"ν. 3469/2006, ν. 4622/2019",άρθρου 37 του Συντάγματος
1,FEK-2021-Tefxos A-00002-downloaded -21_12_2023...,ΕΦΗΜΕΡΙ∆Α TΗΣ ΚΥΒΕΡΝΗΣΕΩΣ Τεύχος A’ 2,05.01.2021,προεδρικο διαταγμα,2.0,"Διορισμός Υπουργών, Αναπληρωτών Υπουργών και Υ...",π.δ. 29/2018,"ν. 3469/2006, ν. 4622/2019",άρθρου 37 του Συντάγματος
2,FEK-2021-Tefxos A-00003-downloaded -21_12_2023...,ΕΦΗΜΕΡΙ∆Α TΗΣ ΚΥΒΕΡΝΗΣΕΩΣ Τεύχος A’ 3,06.01.2021,προεδρικο διαταγμα,3.0,Μεταφορά υπηρεσιών και αρμοδιοτήτων μετα-ξύ Υπ...,,,
3,FEK-2021-Tefxos A-00006-downloaded -21_12_2023...,ΕΦΗΜΕΡΙ∆Α TΗΣ ΚΥΒΕΡΝΗΣΕΩΣ Τεύχος A’ 6,15.01.2021,,NaN,Η ΠΡΟΕΔΡΟΣΤΗΣ ΕΛΛΗΝΙΚΗΣ ΔΗΜΟΚΡΑΤΙΑΣ,π.δ. 29/2018,ν. 3469/2006,
4,FEK-2021-Tefxos A-00007-downloaded -21_12_2023...,ΕΦΗΜΕΡΙ∆Α TΗΣ ΚΥΒΕΡΝΗΣΕΩΣ Τεύχος A’ 7,15.01.2021,προεδρικο διαταγμα,6.0,Οργανισμός του Υπουργείου Δικαιοσύνης.,"π.δ. 96/2017, π.δ. 63/2005, π.δ. 29/2018, π.δ....","ν. 4412/2016, ν. 3469/2006, ν. 2331/1995, ν. 4...","άρθρου 87 του Συντάγματος, άρθρου 91 του Συντά..."
5,FEK-2021-Tefxos A-00008-downloaded -21_12_2023...,ΕΦΗΜΕΡΙ∆Α TΗΣ ΚΥΒΕΡΝΗΣΕΩΣ Τεύχος A’ 8,15.01.2021,νομος,4766.0,Έγκριση σχεδίων συμβάσεων στον τομέα της άμυνας.,π.δ. 29/2018,"ν. 3469/2006, ν. 3978/2011",
6,FEK-2021-Tefxos A-00009-downloaded -21_12_2023...,ΕΦΗΜΕΡΙ∆Α TΗΣ ΚΥΒΕΡΝΗΣΕΩΣ Τεύχος A’ 9,21.01.2021,νομος,4767.0,Καθορισμός του εύρους της αιγιαλίτιδας ζώνης σ...,π.δ. 107/2020,"ν. 2321/1995, ν. 2538/1997",
7,FEK-2021-Tefxos A-00010-downloaded -21_12_2023...,ΕΦΗΜΕΡΙ∆Α TΗΣ ΚΥΒΕΡΝΗΣΕΩΣ Τεύχος A’ 10,25.01.2021,προεδρικο διαταγμα,7.0,Τροποποίηση διατάξεων του π.δ. 54/2019 «Προ-έλ...,"π.δ. 29/2018, π.δ. 54/2019","ν. 3469/2006, ν. 4622/2019, ν. 4270/2014, ν. 4...",
8,FEK-2021-Tefxos A-00012-downloaded -21_12_2023...,ΕΦΗΜΕΡΙ∆Α TΗΣ ΚΥΒΕΡΝΗΣΕΩΣ Τεύχος A’ 12,27.01.2021,προεδρικο διαταγμα,8.0,Αποδοχή Τροποποιήσεων του 2018 στη Σύμβα-ση Να...,"π.δ. 63/2005, π.δ. 29/2018, π.δ. 13/2018, π.δ....","ν. 4078/2012, ν. 3469/2006, ν. 4150/2013, ν. 4...",
9,FEK-2021-Tefxos A-00013-downloaded -21_12_2023...,ΕΦΗΜΕΡΙ∆Α TΗΣ ΚΥΒΕΡΝΗΣΕΩΣ Τεύχος A’ 13,29.01.2021,προεδρικο διαταγμα,9.0,Τροποποίηση διατάξεων του π.δ. 24/1997 «Ιε-ραρ...,,,


In [18]:
results.to_excel('feks_20231221.xlsx',index=False)

: 